In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../data/DorianGray/character-tagging/DorianGray.entities', delimiter='\t')
df

,COREF,start_token,end_token,prop,cat,text
0,358,0,1,NOM,FAC,The studio
1,359,22,23,NOM,FAC,the garden
2,360,63,63,PRON,PER,he
3,360,71,71,PRON,PER,his
4,112,77,79,PROP,PER,Lord Henry Wotton
...,...,...,...,...,...,...
14250,3074,96002,96004,NOM,PER,a dead man
14251,3074,96014,96014,PRON,PER,his
14252,3074,96017,96017,PRON,PER,He
14253,1831,96032,96032,PRON,PER,they


In [3]:
df_characters = df[df.cat == 'PER']
df_characters

,COREF,start_token,end_token,prop,cat,text
2,360,63,63,PRON,PER,he
3,360,71,71,PRON,PER,his
4,112,77,79,PROP,PER,Lord Henry Wotton
5,361,115,115,PRON,PER,theirs
6,112,156,156,PRON,PER,him
...,...,...,...,...,...,...
14250,3074,96002,96004,NOM,PER,a dead man
14251,3074,96014,96014,PRON,PER,his
14252,3074,96017,96017,PRON,PER,He
14253,1831,96032,96032,PRON,PER,they


In [4]:
len(df_characters.groupby(df.COREF))

1686

In [5]:
characters = {}

for coref_id, coref_df in df_characters.groupby(df.COREF):
    try:
        proper_noun_df = coref_df[coref_df.prop == 'PROP']
        if not proper_noun_df.empty:
            name = proper_noun_df.text[proper_noun_df.text.str.len() == proper_noun_df.text.str.len().max()].unique()[0]
            characters[coref_id] = name
    except:
        continue

characters

{3: 'The Grosvenor',
 11: 'Paris',
 31: 'Berwick',
 88: 'young Perth',
 91: 'Selby',
 94: 'Venice',
 112: 'Lord Henry Wotton',
 113: 'Basil Hallward',
 115: 'Mr. Dorian Gray',
 116: 'Poor Lady Brandon',
 117: 'Conscience',
 118: 'Mr. Hallward',
 119: 'poor Southwark',
 120: 'the Venetians',
 121: 'Antinous',
 122: 'Greek',
 123: 'Agnew',
 124: 'One',
 125: 'Lord Goodbody',
 126: 'Lady Agatha',
 127: 'Parker',
 128: 'Mind',
 129: 'Schumann',
 130: 'Aunt Agatha',
 131: 'Good God',
 132: 'June',
 133: 'White',
 134: 'Lord Fermor',
 135: 'Society',
 136: 'Isabella',
 137: 'Uncle George',
 138: 'the last Lord Kelso',
 139: 'Lady Margaret Devereux',
 140: 'Egad',
 141: 'the jarvies',
 143: 'Carlington',
 144: 'the Americans',
 145: 'Eve',
 146: 'Worlds',
 147: 'Plato',
 148: 'Buonarotti',
 149: 'Sir Thomas Burdon',
 150: 'Mr. Erskine of Treadley',
 151: 'Mrs. Vandeleur',
 152: 'Lord Faudel',
 153: 'Duchess',
 154: 'Humour',
 155: 'Facts',
 156: 'Omar',
 157: 'Willis',
 158: 'the English',
 1

In [6]:
from collections import defaultdict
import string


merged_idxs = set()
merged_coref_ids = defaultdict(list)

female_titles = [
    'Mrs.',
    'Lady',
    'Miss',
    'Aunt',
    'Duchess',
]

male_titles = [
    'Sir',
    'Mr.',
    'Lord',
    'Uncle',
]

neutral_titles_and_starters = [
    'Dr.',
    'Dear',
    'Poor',
    'Old',
]


# Merge characters based on titles
for i, (coref_id, base_name) in enumerate(characters.items()):
    
    if i in merged_idxs:
        continue
    
    try:
        base_split_name = base_name.split()

        base_titles = []
        base_gender = 'neutral'
        while base_split_name[0] in male_titles + female_titles + neutral_titles_and_starters:

            if base_split_name[0] in male_titles:
                base_gender = 'male'
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            elif base_split_name[0] in female_titles:
                base_gender = 'female'
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            else:
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            
        base_forename = base_split_name[0]
        base_surname = base_split_name[-1]
            

        for j, (coref_id, other_name) in enumerate(characters.items()):
            
            if j in merged_idxs:
                continue
            
            if j <= i:
                continue
            other_split_name = other_name.split()
            
            other_titles = []
            other_gender = 'neutral'
            while other_split_name[0] in male_titles + female_titles + neutral_titles_and_starters:

                if other_split_name[0] in male_titles:
                    other_gender = 'male'
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])
                elif other_split_name[0] in female_titles:
                    other_gender = 'female'
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])
                else:
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])

            other_forename = other_split_name[0]
            other_surname = other_split_name[-1]
            
            if base_gender == 'male' and other_gender == 'female' or base_gender == 'female' and other_gender == 'male':
                continue
            
            if other_forename != other_surname and base_forename != base_surname:
                if base_forename != other_forename or base_surname != other_surname:
                    continue
                else:
                    merged_idxs.add(j)
                    merged_coref_ids[i].append(j)
                    print(f'Merging1: {base_name} -> {other_name}')
                    continue
            
            # Otherwise gender is the same/neutral, if the forenames match we should merge
            
            if other_forename == base_forename:
                merged_idxs.add(j)
                merged_coref_ids[i].append(j)
                print(f'Merging2: {base_name} -> {other_name}')
    except Exception as e:
        pass

Merging2: Lady Agatha -> Aunt Agatha
Merging2: Lady Henry -> Henry VIII
Merging2: Lady Henry -> Henry II
Merging2: Miss Sibyl Vane -> Poor Sibyl
Merging2: Dear Basil -> Poor Basil
Merging2: Sir Geoffrey Clouston -> Poor Geoffrey
Merging2: Hetty Merton -> Poor Hetty
Merging2: Monsieur -> Monsieur


In [7]:
df_merged = df_characters[df_characters['COREF'].isin(list(characters.keys()))]
df_merged.COREF.unique().size

248

In [8]:
for coref_id, merged_coref_list in merged_coref_ids.items():
    for merged_coref_id in merged_coref_list:
        df_merged.loc[df_merged["COREF"] == merged_coref_id, "COREF"] = coref_id
    
df_merged.COREF.unique().size

244

In [9]:
min_mentions = 10

df_filtered = df_merged.groupby('COREF').filter(lambda x: len(x) > min_mentions)
df_filtered.COREF.unique().size

35

In [10]:
df_filtered

,COREF,start_token,end_token,prop,cat,text
4,112,77,79,PROP,PER,Lord Henry Wotton
6,112,156,156,PRON,PER,him
14,113,289,290,PROP,PER,Basil Hallward
27,113,396,396,PROP,PER,Basil
29,112,408,409,PROP,PER,Lord Henry
...,...,...,...,...,...,...
14196,307,95668,95668,PRON,PER,he
14197,307,95674,95674,PRON,PER,He
14215,115,95814,95816,PROP,PER,Mr. Dorian Gray
14232,268,95885,95885,PROP,PER,Francis


In [11]:
df_filtered['character_name'] = df_filtered.COREF.map(lambda x: characters[x])

In [12]:
df_filtered

,COREF,start_token,end_token,prop,cat,text,character_name
4,112,77,79,PROP,PER,Lord Henry Wotton,Lord Henry Wotton
6,112,156,156,PRON,PER,him,Lord Henry Wotton
14,113,289,290,PROP,PER,Basil Hallward,Basil Hallward
27,113,396,396,PROP,PER,Basil,Basil Hallward
29,112,408,409,PROP,PER,Lord Henry,Lord Henry Wotton
...,...,...,...,...,...,...,...
14196,307,95668,95668,PRON,PER,he,Alan Campbell
14197,307,95674,95674,PRON,PER,He,Alan Campbell
14215,115,95814,95816,PROP,PER,Mr. Dorian Gray,Mr. Dorian Gray
14232,268,95885,95885,PROP,PER,Francis,Francis Osborne


In [13]:
df_filtered.to_csv('../data/DorianGray/processed_characters.csv')

PermissionError: [Errno 13] Permission denied: '../data/DorianGray/processed_characters.csv'